In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

n_users = 500
users = np.arange(1, n_users + 1)

events = []

for user in users:
    # 1. view_product (herkes görür)
    events.append([user, "view_product"])
    
    # 2. add_to_cart (%60)
    if np.random.rand() < 0.6:
        events.append([user, "add_to_cart"])
        
        # 3. checkout (%70)
        if np.random.rand() < 0.7:
            events.append([user, "checkout"])
            
            # 4. purchase (%50)
            if np.random.rand() < 0.5:
                events.append([user, "purchase"])

df = pd.DataFrame(events, columns=["user_id", "event"])
df.head()


In [ ]:
df.to_csv("ecommerce_funnel_events.csv", index=False)


In [ ]:
df["event"].value_counts()


In [ ]:
funnel_order = [
    "view_product",
    "add_to_cart",
    "checkout",
    "purchase"
]


In [ ]:
funnel_counts = (
    df[df["event"].isin(funnel_order)]
    .groupby("event")["user_id"]
    .nunique()
    .reindex(funnel_order)
)

funnel_counts


In [ ]:
funnel_df = funnel_counts.reset_index()
funnel_df.columns = ["step", "users"]
funnel_df


In [ ]:
funnel_df["conversion_rate"] = funnel_df["users"] / funnel_df["users"].shift(1)
funnel_df["drop_off_rate"] = 1 - funnel_df["conversion_rate"]

funnel_df


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.bar(funnel_df["step"], funnel_df["users"])
plt.xlabel("Funnel Step")
plt.ylabel("Number of Users")
plt.title("User Drop-off Across Purchase Funnel")
plt.show()
